In [20]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Using a popular classification library
iris = load_iris()
data = pd.DataFrame(data=iris.data, columns=iris.feature_names)
data['species'] = iris.target

In [33]:
class RandomForestClassifier:
    def __init__(self, n_estimators: int = 10, max_depth: int = 5, min_samples_leaf: int = 3, max_features: int | None = None):
        self.n_estimators = n_estimators # Number of decision trees
        self.max_depth = max_depth # Max depth of each decision tree
        self.min_samples_leaf = min_samples_leaf # Min samples for a node to be declared as a leaf
        self.max_features = max_features # Optional input which restricts the max number of features a tree can be trained on
        self.trees: list = [] # Stores the trees and corresponding features used


    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        '''
        Trains the random forest by training multiple decision trees.

        For each decision tree, choose a random sample of data points
        and a random set of features to train it on. This ensures that
        no feature is dominant and reduces overfitting. The samples
        shared between trees can overlap (choosing with replacement).
        '''

        self.trees = [] # Initializing an empty list
        n_samples, n_features = X.shape
        self.max_features = self.max_features or int(np.sqrt(n_features)) # If no value is given, reverts to default (sqrt)

        for _ in range(self.n_estimators):
            # TODO 1: Take random samples of the dataset from here (Bootstrapping)
            # YOUR CODE HERE
            ranrows=np.random.choice(range(n_samples),size=n_samples,replace=True)
            X_sample, y_sample = X[ranrows,:], y[ranrows] # Fill with appropriate values
            tree = DecisionTreeClassifier(max_depth=self.max_depth, min_samples_leaf=self.min_samples_leaf)
            # TODO 2: Train the decision tree using random features
            # YOUR CODE HERE
            numfe=np.random.randint(1,self.max_features)
            tree_features = np.random.choice(range(n_features),size=numfe,replace=False) # Indices of the required features
            X_sample_subfeatures = X_sample[:,tree_features] # Fill with appropriate value (remember to use max_features)
            # Fit the tree using the subset of data
            tree.fit(X_sample_subfeatures,y_sample)
            self.trees.append((tree, tree_features))


    def predict(self, X: np.ndarray) -> np.ndarray:
        '''
        Predicts class labels for the given input data.

        Pass the given data through all decision trees in self.trees,
        then choose the class that is given by majority of the trees.
        Use only the features that the tree was trained on -
        (the feature indices are also stored in the list self.trees)
        '''
        # TODO 3: Complete the predict function
        # YOUR CODE HERE
        answers=list()
        for tree in self.trees:
            x_test=X[:,tree[1]]
            midstep=tree[0].predict(x_test)
            answers.append(midstep)
        return answers

In [39]:
# TODO 3: Fill in appropriate values, large value of estimators or depth could lead to
# overfitting, while large values of max_features and min_samples_leaf could lead to underfitting
model = RandomForestClassifier(n_estimators=10, max_depth=5, min_samples_leaf=3, max_features=4)
X, y = data.drop(columns='species').to_numpy(),data['species'].to_numpy()# Use the iris data set that was initialized in the first code block

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = np.mean(y_pred == y_test)
print(f"Your model has an accuracy of {100 * accuracy}%.")

Your model has an accuracy of 96.66666666666667%.
